### Preparar ambiente

In [1]:
import sys
!{sys.executable} -m pip install PyAthena

### Conectar con pyathena

In [2]:
from pyathena.pandas.cursor import PandasCursor
from pyathena import connect
import pandas as pd
import time
from datetime import datetime
# agregar directorio en el cual se almacenan las tablas que se van a consultar
directorio = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/athena-results/'
# base de datos
bd = 'graphdata' # Database in Glue
cursor = connect(s3_staging_dir = directorio, region_name = 'us-east-1', schema_name = bd, cursor_class = PandasCursor).cursor()

In [3]:
import pandas as pd
import numpy as np
import calendar
import boto3
s3 = boto3.resource('s3')
bucket='results-and-tables-graphdata'

In [4]:
import os
import pandas as pd
import zipfile
import shutil
from io import StringIO
import io

### Declaración funciones para Impo-Expo s3

In [5]:
def write_dataframe_to_csv_on_s3(dataframe, bucket, filename):
    """ Write a dataframe to a CSV on S3 """
    # Create buffer
    csv_buffer = StringIO()
    # Write dataframe to buffer
    dataframe.to_csv(csv_buffer, sep=";" , line_terminator='\n')
    # Create S3 object
    s3_resource = boto3.resource("s3")
    # Write buffer to S3 object
    s3_resource.Object(bucket, filename).put(Body=csv_buffer.getvalue())
    print("Writing {} records to {}".format(len(dataframe), filename))
    print("S3")
    print(dataframe.shape)
    

In [6]:
def write_dataframe_to_excel_on_s3(dataframe, bucket, filename):
    """ Write a dataframe to a excel on S3 """
    with io.BytesIO() as output:
        with pd.ExcelWriter(output, engine='xlsxwriter') as writer:
            dataframe.to_excel(writer,index=False)
        data = output.getvalue()

    s3 = boto3.resource('s3')
    s3.Bucket(bucket).put_object(Key=filename, Body=data)
    

In [7]:
def import_dataframe_csv_from_s3(bucket, filename):
    client = boto3.client('s3')
    bucket_name = bucket
    object_key = filename
    csv_obj = client.get_object(Bucket=bucket_name, Key=object_key)
    body = csv_obj['Body']
    csv_string = body.read().decode('utf-8')
    df = pd.read_csv(StringIO(csv_string),sep=";")
    return df

In [8]:
def import_dataframe_excel_from_s3(bucket, filename):
    client = boto3.client('s3')
    bucket_name = bucket
    object_key = filename
    obj = client.get_object(Bucket=bucket_name, Key=object_key)
    data = obj['Body'].read()
    df = pd.read_excel(io.BytesIO(data), encoding='utf-8')
    return df

In [9]:
def write_file_to_s3(bucket,file_local,file_s3):
    s3.Bucket(bucket).upload_file(file_local,file_s3)

### Importar tabla de Athena

In [10]:
#toda la base
min_distancia = 1 #km
min_pings_dia = 10
dfs_indicadores = []

tabla_usuarios_distancia =  f'historico_usuarios'    
query_filtro = (f"SELECT t.year, "
                    "t.month, "
                    "t.day, "
                    "t.iso_country_code, "
                    "COUNT(*) as n_users, "
                    f"SUM(CASE WHEN t.distancia_recorrida >= {min_distancia} then 1 ELSE 0 END) as n_users_mas1km "
                    f"FROM ("
                        "SELECT * "
                        f"FROM {tabla_usuarios_distancia} "
                        f"WHERE ({tabla_usuarios_distancia}.n_obs >= {min_pings_dia})) AS t "
                    "GROUP BY t.year, t.month, t.day, t.iso_country_code")

df = cursor.execute(query_filtro).as_pandas()
df['%MasDe1km'] = df['n_users_mas1km'] / df['n_users']*100
dfs_indicadores.append(df)
df_by_day2athena = pd.concat(dfs_indicadores)  

In [11]:
df_by_day2at=df_by_day2athena
df_by_day2at=pd.DataFrame(df_by_day2at)

In [12]:
df_by_day2at['month']=df_by_day2at['month'].astype('int')
df_by_day2at['day']=df_by_day2at['day'].astype('int')
df_by_day2at["day"]=df_by_day2at["day"].map("{:02}".format)
df_by_day2at["key4"]=df_by_day2at['month'].astype(str)+ df_by_day2at["day"].astype(str)
df_by_day2at['key4']=df_by_day2at['key4'].astype('int')
df_by_day2at=df_by_day2at.rename(columns={'month':'Mes','day':'Dia','%MasDe1km':'%MasDe1km_new','n_users_mas1km':'MasDe1km'})
df_by_day2at=df_by_day2at.loc[:,['Mes','Dia','key4','MasDe1km','n_users','%MasDe1km_new','iso_country_code']]
df_by_day2at['Dia']=df_by_day2at['Dia'].astype('int')

In [13]:
df_by_day2at=df_by_day2at.sort_values(by=['iso_country_code','Mes','Dia'])

In [14]:
df_by_day2at.to_csv("serie_bruta_nacional.csv",sep=";")

### Indices Nacionales

In [15]:
paises = ['DO','SV','CR','BO','CL','EC','AR','VE','GT','HN','NI','PA','PY','PE','UY','BZ','GY','JM','TT','CO','BR','MX'] 
#paises = ['CL']
for pais in paises:
    print(f"Pais: {pais}")
    
    df_by_day=df_by_day2at[df_by_day2at['iso_country_code']==pais]
    df_by_day=df_by_day.rename(columns={'iso_country_code':'country'})
    df_by_day=df_by_day.sort_values(by=['Mes','Dia'])
    
    ##computar indice suavizado
    
    ##Baseline    
    year="2020"
    df_by_day["fecha"] = year + "-" + df_by_day['Mes'].astype(int).astype(str) + "-" + df_by_day["Dia"].astype(int).astype(str)
    df_by_day['weekday'] = pd.to_datetime(df_by_day['fecha']).apply(lambda x: x.weekday())
    df_by_day["weekday2"]=0 
    df_by_day["weekday2"][df_by_day["weekday"]==6]="Domingo"       
    df_by_day["weekday2"][df_by_day["weekday"]==5]="Sabado" 
    df_by_day["weekday2"][(df_by_day["weekday"]!=5) & (df_by_day["weekday"]!=6)]="Semana" 
    df_by_day["key"]=df_by_day["weekday2"].astype(str)+df_by_day["country"].astype(str)
    reg=df_by_day["country"].unique()
    dia8=df_by_day[(df_by_day["Mes"]==2) & (df_by_day["Dia"]==8)].groupby(["country"])["%MasDe1km_new"].sum()
    dia8=pd.DataFrame(dia8)
    dia8["fecha"]="Domingo"
    
    #dia8["country"]=reg
    dia8=dia8.reset_index()
    dia7=df_by_day[(df_by_day["Mes"]==2) & (df_by_day["Dia"]==7)].groupby(["country"])["%MasDe1km_new"].sum()
    dia7=pd.DataFrame(dia7)
    dia7["fecha"]="Sabado"
    
    #dia7["country"]=reg
    dia7=dia7.reset_index()
    diasem=df_by_day[(df_by_day["Mes"]==2) & (df_by_day['Dia'].isin(['5', '6', '9', '10', '11']))].groupby(["country"])["%MasDe1km_new"].mean()
    diasem=pd.DataFrame(diasem)
    diasem["fecha"]="Semana"
    
    #diasem["country"]=reg
    diasem=diasem.reset_index()
    match=diasem.append(dia8)
    match=match.append(dia7)
    match["key"]=match["fecha"].astype(str)+match["country"].astype(str)
    match=match.rename(columns={"%MasDe1km_new": "Baseline"})
    match=match.loc[:,["key","Baseline"]]
    df_by_day=df_by_day.merge(match,how='left',on=['key'])
    
    ##Ratio
    df_by_day["ratio"]=df_by_day['%MasDe1km_new']/df_by_day['Baseline']
    
    ##por excepcion del día conflictivo
    df_by_day["ratio_lag1"]=df_by_day.groupby(["country"])["ratio"].shift(1)
    df_by_day["ratio_f1"]=df_by_day.groupby(["country"])["ratio"].shift(-1)
    
    ##Suavizado
    df_by_day["ratio_sua"]=(df_by_day["ratio"]+df_by_day["ratio_lag1"]+df_by_day["ratio_f1"])/3
    
    ##Indice
    df_by_day["FracMas1km"]=df_by_day["ratio_sua"]-1
    
    ##key para tabla pivot
    df_by_day["Dia"]=df_by_day["Dia"].astype(int)
    df_by_day["Dia"]=df_by_day.Dia.map("{:02}".format)
    df_by_day["key4"]=df_by_day['Mes'].astype(int).astype(str)+ df_by_day["Dia"].astype(str)
    
    ##genero dataframe para exportar
    df_by_day2=df_by_day.loc[:,["Mes", "Dia","key4", "country", "MasDe1km", "n_users",'%MasDe1km_new',"FracMas1km"]]
    
    ##tabla pivot para publicar
    table_piv = pd.pivot_table(df_by_day, values='FracMas1km', index=['country'],
                    columns=['key4'], aggfunc=np.sum)
    table_piv=table_piv[table_piv.columns[1:-1]]
    table_piv = table_piv*100
    table_piv = table_piv.round(2)
    table_piv.index.name="cod"
    
    ruta=f'graphdata-mobility-public/indicador_por_pais_dashboard/{pais}_nacion_analisis.csv'
    write_dataframe_to_csv_on_s3(df_by_day2, 'iadbprod-csd-hub-analyticaldata', ruta)
    
    ruta2=f'graphdata-mobility-public/indicador_por_pais_dashboard/{pais}_nacion.csv'
    write_dataframe_to_csv_on_s3(table_piv, 'iadbprod-csd-hub-analyticaldata', ruta2)  

Pais: DO


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Writing 120 records to graphdata-mobility-public/indicador_por_pais_dashboard/DO_nacion_analisis.csv
S3
(120, 8)
Writing 1 records to graphdata-mobility-public/indicador_por_pais_dashboard/DO_nacion.csv
S3
(1, 118)
Pais: SV


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Writing 120 records to graphdata-mobility-public/indicador_por_pais_dashboard/SV_nacion_analisis.csv
S3
(120, 8)
Writing 1 records to graphdata-mobility-public/indicador_por_pais_dashboard/SV_nacion.csv
S3
(1, 118)
Pais: CR


...

Writing 120 records to graphdata-mobility-public/indicador_por_pais_dashboard/CO_nacion_analisis.csv
S3
(120, 8)
Writing 1 records to graphdata-mobility-public/indicador_por_pais_dashboard/CO_nacion.csv
S3
(1, 118)
Pais: BR


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Writing 120 records to graphdata-mobility-public/indicador_por_pais_dashboard/BR_nacion_analisis.csv
S3
(120, 8)
Writing 1 records to graphdata-mobility-public/indicador_por_pais_dashboard/BR_nacion.csv
S3
(1, 118)
Pais: MX


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Writing 120 records to graphdata-mobility-public/indicador_por_pais_dashboard/MX_nacion_analisis.csv
S3
(120, 8)
Writing 1 records to graphdata-mobility-public/indicador_por_pais_dashboard/MX_nacion.csv
S3
(1, 118)
